In [308]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [426]:
df=pd.read_csv('TransactionRecord-v-return added.csv')

**Filtering Out Missing Data**

In [427]:
df=df.dropna()

**Checking Duplicates**

In [429]:
sum(df.duplicated())#no duplicates found

0

**Rename the columns(for ease of manipulation)**

In [430]:
df=df.rename(columns={'trd_date':'Date','fund_id':'FID1',
                  'unit_count':'Units','gender':'Gender',
                  'TrailerRate':'TRate','FundRiskScore':'FRSC','FundName':'FName'})

**Drop Unnecessary Columns**

In [431]:
del df['FName']

In [432]:
del df['intermediaryid']

In [433]:
df.columns

Index(['Unnamed: 0', 'Date', 'FID1', 'investor_id', 'Units', 'Gender',
       'income', 'education', 'riskscore', 'subscribed', 'opened', 'SCRate',
       'TRate', 'FRSC', 'Price', 'return-N6M', 'return-HYG', 'return-IVV'],
      dtype='object')

**Create a separate table, create dummy variables for Fund Type**

In [434]:
dummies_F1D1=pd.get_dummies(df['FID1'],prefix='FID1')

In [435]:
df=df.join(dummies_F1D1)

Due to the impending "group-by" operation, rename the FID1_xxx columns as "xxx_txn_cnt".

In [436]:
df=df.rename(columns={'FID1_HYG':'HYG_txn_cnt', 'FID1_IVV':'IVV_txn_cnt', 'FID1_N6M':'N6M_txn_cnt'})

In [437]:
df.columns

Index(['Unnamed: 0', 'Date', 'FID1', 'investor_id', 'Units', 'Gender',
       'income', 'education', 'riskscore', 'subscribed', 'opened', 'SCRate',
       'TRate', 'FRSC', 'Price', 'return-N6M', 'return-HYG', 'return-IVV',
       'HYG_txn_cnt', 'IVV_txn_cnt', 'N6M_txn_cnt'],
      dtype='object')

**Create additional field for transaction value, drop entries with 0 transaction value.**

In [438]:
df['Txn_Val']=df['Units']*df['Price']

In [325]:
df = df[df['Txn_Val'] != 0]

**Convert FundRiskScore to numeric**

[Note: This is a shortcut in preparation for the grouping step.  Implicit assumption is that response is linear as we move from L to M to H.]

In [440]:
df.FRSC[df['FRSC']=='H']=3

<ipython-input-440-cb1466e17708>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.FRSC[df['FRSC']=='H']=3


In [441]:
df.FRSC[df['FRSC']=='M']=2

<ipython-input-441-2f775214fe4d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.FRSC[df['FRSC']=='M']=2


In [442]:
df.FRSC[df['FRSC']=='L']=1

<ipython-input-442-d3129b2134f4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.FRSC[df['FRSC']=='L']=1


[Pseudo code] Convert FRSC from L/M/H to numerics 1/2/3.

**Create additional fields about date range, overall return, and return volatility.**

[Issue] 
On a deeper thought, to mimic the actual investor thought process, it may require a regression to a time series model if we want to get better fit.  In that mode, the investment return at the date when an actual transaction has occured (which Duan Ke inserted on Wed) is meaningful.  However, this may be too complex given the time constraints we have.  So I am thinking if there is a workaround without giving up too much predictive power.

[Pseudo code:]

- Count the unique number of trd_date in the data frame
- Observe the max and min value of trd_data in the data frame
- Open up the FundPxVol.csv file and read into a temporary data frame.
- Using that information:
  - Calculate 3 values - N6M_rtn, HYG_rtn, IVV_rtn - as [( Price @ max_date / Price @ min date) - 1] for each of the 3 funds.
  - Calculate 3 values - N6M_vol, HYG_vol, IVV_vol - as sd(Return) between the max_date and min_date for each of the 3 funds.

In [444]:
df['DATE']=pd.to_datetime(df.Date)

In [446]:
df.index=df['DATE']

In [447]:
df_funds=pd.read_csv('FundPxVol (1).csv',date_parser=True)

In [448]:
df_funds.index=pd.to_datetime(df_funds.TradeDate)

In [450]:
df_HYG=df_funds['2019-08-26':'2020-06-04']

In [451]:
vol_HYG=np.std(df_HYG[df_HYG.FundID=='HYG'].Price)

In [452]:
df_N6M=df_funds['2019-08-26':'2020-04-28']

In [453]:
vol_N6M=np.std(df_N6M[df_N6M.FundID=='N6M'].Price)

In [454]:
df_IVV=df_funds['2019-08-26':'2020-07-27']

In [455]:
vol_IVV=np.std(df_IVV[df_IVV.FundID=='IVV'].Price)

In [456]:
N6M_rtn=11.33/11.14

In [457]:
HYG_rtn=83.669998/86.980003

In [458]:
IVV_rtn=324.500000/289.660004

In [459]:
df.columns

Index(['Unnamed: 0', 'Date', 'FID1', 'investor_id', 'Units', 'Gender',
       'income', 'education', 'riskscore', 'subscribed', 'opened', 'SCRate',
       'TRate', 'FRSC', 'Price', 'return-N6M', 'return-HYG', 'return-IVV',
       'HYG_txn_cnt', 'IVV_txn_cnt', 'N6M_txn_cnt', 'Txn_Val', 'DATE'],
      dtype='object')

In [460]:
df['rtn-N6M']=N6M_rtn

In [461]:
df['rtn-HYG']=HYG_rtn

In [462]:
df['rtn-IVV']=IVV_rtn

In [463]:
df['vol_N6M']=vol_N6M

In [464]:
df['vol_HYG']=vol_HYG

In [465]:
df['vol_IVV']=vol_IVV

In [466]:
df.columns

Index(['Unnamed: 0', 'Date', 'FID1', 'investor_id', 'Units', 'Gender',
       'income', 'education', 'riskscore', 'subscribed', 'opened', 'SCRate',
       'TRate', 'FRSC', 'Price', 'return-N6M', 'return-HYG', 'return-IVV',
       'HYG_txn_cnt', 'IVV_txn_cnt', 'N6M_txn_cnt', 'Txn_Val', 'DATE',
       'rtn-N6M', 'rtn-HYG', 'rtn-IVV', 'vol_N6M', 'vol_HYG', 'vol_IVV'],
      dtype='object')

**Developing the response variables**

[Note] Different response variables (AUM per investor, AUM per transaction per investor, transaction frequency per investor, etc) will require a different grouping approach.

[Note 2] The codes for each response variable can be run separately. 

[Note 3] Refer to the business question that we are trying to answer via visualization.  Currently, these data cleaning script are using the full data set. That is good for business question #2 when we are trying to say, within the date range chosen by the user, how much of the variance in the response variable can be explained by the different Xs.  However, for the forecast question (biz question #4 in the slides), if we want to test on unseen data, then then "training set" (which will get further split between train vs validation during cross-validation) may need to cut of at an earlier date.  Would be useful, somewhere at the start of this script, to put in two input parameters for us to set the date range we want to run this data cleaning script for.

**Response variable 1) Change in AUM per investor**

**Apply "Groupby" based on investor_id**

In [467]:
keys=[df['investor_id']]

[Note:] Since we are dealing with change in AUM, any transactions related to 2019/8/25 should be excluded from grouping.

In [469]:
grouped=df[df.Date != '2019-8-25'].groupby(keys)

In [470]:
df_aggTxnval=(grouped.sum()).iloc[:,[16]]

[Pseudo code] Keep only investor_id and Txn_Val of df_iid because it does not make sense to apply .sum() to the rest of the columns.  

[Pseudo code] Check the total row count of df_iid.

In [361]:
df_aggTxnval.describe()

,Txn_Val
count,950.000000
mean,30786.042814
std,59941.149876
min,-526.772856
25%,4536.515884
50%,12118.000112
75%,30574.773680
max,806875.041052


[Pseudo code] Using this reduced df_iid as an anchor, perform left join with df on investor_id so that the following 5 columns can be added to df_iid: income, education, riskscore, subscribed, opened.  Check that the row count remain unchanged after joining, or remove duplication to keep row count constant.

In [472]:
df_iid=pd.merge(df_aggTxnval,df,on=['investor_id'])

In [473]:
df_iid.columns

Index(['investor_id', 'Txn_Val_x', 'Unnamed: 0', 'Date', 'FID1', 'Units',
       'Gender', 'income', 'education', 'riskscore', 'subscribed', 'opened',
       'SCRate', 'TRate', 'FRSC', 'Price', 'return-N6M', 'return-HYG',
       'return-IVV', 'HYG_txn_cnt', 'IVV_txn_cnt', 'N6M_txn_cnt', 'Txn_Val_y',
       'DATE', 'rtn-N6M', 'rtn-HYG', 'rtn-IVV', 'vol_N6M', 'vol_HYG',
       'vol_IVV'],
      dtype='object')

In [474]:
del df_iid['Unnamed: 0']

In [475]:
del df_iid['Txn_Val_y']

[Pseudo code] Then, add in 6 more columns based on the values calculated earlier - N6M_rtn, HYG_rtn, IVV_rtn, N6M_vol, HYG_vol, IVV_vol.

In [363]:
##Already Added to the orginal dataframe

[Pseudo code] Create df_iid1 =grouped.mean().  Keep only 3 columns where aggregation as mean makes more sense - SCRate, TRate, FRSC.

In [476]:
fill_mean=lambda g:g.mean()

In [477]:
df_iid1=grouped.apply(fill_mean)

In [478]:
df_iid1.describe()

,Unnamed: 0,investor_id,Units,income,riskscore,subscribed,opened,SCRate,TRate,FRSC,...,HYG_txn_cnt,IVV_txn_cnt,N6M_txn_cnt,Txn_Val,rtn-N6M,rtn-HYG,rtn-IVV,vol_N6M,vol_HYG,vol_IVV
count,993.000000,993.000000,993.000000,9.930000e+02,993.000000,993.000000,993.000000,993.000000,993.000000,993.000000,...,993.000000,993.000000,993.000000,993.000000,9.930000e+02,9.930000e+02,9.930000e+02,9.930000e+02,9.930000e+02,9.930000e+02
mean,3339.683713,55309.830816,167.423195,1.651977e+05,3.651561,0.319235,0.154079,0.025889,0.001074,2.049160,...,0.386735,0.331212,0.282052,4647.239909,1.017056e+00,9.619452e-01,1.120279e+00,3.673701e-01,4.435599e+00,2.210263e+01
std,1182.300919,25914.321584,513.380631,2.058490e+05,1.367627,0.466415,0.361206,0.017173,0.001738,0.605335,...,0.222174,0.319084,0.325701,9909.073058,1.660361e-14,1.361498e-14,1.532880e-14,5.696926e-15,1.037980e-13,3.068649e-13
min,206.000000,10051.000000,-0.890814,3.491008e+03,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,-131.693214,1.017056e+00,9.619452e-01,1.120279e+00,3.673701e-01,4.435599e+00,2.210263e+01
25%,2479.333333,33369.000000,7.958232,5.532879e+04,3.000000,0.000000,0.000000,0.000000,0.000000,1.600000,...,0.250000,0.000000,0.000000,591.755105,1.017056e+00,9.619452e-01,1.120279e+00,3.673701e-01,4.435599e+00,2.210263e+01
50%,3368.888889,54961.000000,30.450733,1.042818e+05,4.000000,0.000000,0.000000,0.031250,0.000000,2.000000,...,0.400000,0.200000,0.200000,1678.466633,1.017056e+00,9.619452e-01,1.120279e+00,3.673701e-01,4.435599e+00,2.210263e+01
75%,4193.142857,78429.000000,106.807256,2.059554e+05,5.000000,1.000000,0.000000,0.040909,0.003000,2.636364,...,0.500000,0.636364,0.444444,4530.638466,1.017056e+00,9.619452e-01,1.120279e+00,3.673701e-01,4.435599e+00,2.210263e+01
max,6077.200000,99642.000000,9015.819277,2.598062e+06,5.000000,1.000000,1.000000,0.050000,0.005000,3.000000,...,1.000000,1.000000,1.000000,138876.279985,1.017056e+00,9.619452e-01,1.120279e+00,3.673701e-01,4.435599e+00,2.210263e+01


In [479]:
df_iid1=df_iid1.iloc[:,[8,9,7]]

[Pseudo code] Using df_iid as anchor, perform left join with df_iid1 on investor_id.  This will patch in the 3 newly-created columns.  Confirm that row count of df_iid remains unchanged. 

In [480]:
df_sn1=pd.merge(df_iid,df_iid1,on='investor_id')

In [481]:
df_sn1.columns

Index(['investor_id', 'Txn_Val_x', 'Date', 'FID1', 'Units', 'Gender', 'income',
       'education', 'riskscore', 'subscribed', 'opened', 'SCRate_x', 'TRate_x',
       'FRSC_x', 'Price', 'return-N6M', 'return-HYG', 'return-IVV',
       'HYG_txn_cnt', 'IVV_txn_cnt', 'N6M_txn_cnt', 'DATE', 'rtn-N6M',
       'rtn-HYG', 'rtn-IVV', 'vol_N6M', 'vol_HYG', 'vol_IVV', 'TRate_y',
       'FRSC_y', 'SCRate_y'],
      dtype='object')

In [563]:
del df_sn1['SCRate_x']

In [564]:
del df_sn1['TRate_x']

In [565]:
del df_sn1['FRSC_x']

In [566]:
df_sn1=df_sn1.join(pd.get_dummies(df_sn1['Gender'],prefix='gen'))

In [568]:
del df_sn1['Gender']

In [569]:
df_sn1=df_sn1.join(pd.get_dummies(df_sn1['education'],prefix='ed'))

In [570]:
del df_sn1['education']

In [571]:
df_sn1['subscribed']=df_sn1['subscribed'].apply(lambda x:1 if x else 0)

In [572]:
df_sn1.columns

Index(['investor_id', 'HYG_txn_cnt_x', 'IVV_txn_cnt_x', 'N6M_txn_cnt_x',
       'Date', 'FID1', 'Units', 'income', 'riskscore', 'subscribed', 'opened',
       'Price', 'return-N6M', 'return-HYG', 'return-IVV', 'DATE', 'rtn-N6M',
       'rtn-HYG', 'rtn-IVV', 'vol_N6M', 'vol_HYG', 'vol_IVV', 'TRate_y',
       'FRSC_y', 'SCRate_y', 'gen_F', 'gen_M', 'ed_Deg or abv',
       'ed_H.Sch/Dip.', 'ed_Sec. or below'],
      dtype='object')

In [493]:
df_sn1.head()

,investor_id,Txn_Val_x,Date,FID1,Units,income,riskscore,subscribed,opened,Price,...,vol_HYG,vol_IVV,TRate_y,FRSC_y,SCRate_y,gen_F,gen_M,ed_Deg or abv,ed_H.Sch/Dip.,ed_Sec. or below
0,10051,23473.925401,2019/8/26,HYG,30.550818,324465.7233,3,0,0,86.980003,...,4.435599,22.102633,0.0,1.5,0.03,0,1,1,0,0
1,10051,23473.925401,2019/12/5,HYG,0.000000,324465.7233,3,0,0,86.809998,...,4.435599,22.102633,0.0,1.5,0.03,0,1,1,0,0
2,10051,23473.925401,2019/12/16,N6M,-238.537721,324465.7233,3,0,0,11.040000,...,4.435599,22.102633,0.0,1.5,0.03,0,1,1,0,0
3,10051,23473.925401,2020/3/2,HYG,120.453956,324465.7233,3,0,0,86.309998,...,4.435599,22.102633,0.0,1.5,0.03,0,1,1,0,0
4,10051,23473.925401,2020/3/25,HYG,0.000000,324465.7233,3,0,0,73.470001,...,4.435599,22.102633,0.0,1.5,0.03,0,1,1,0,0


In [371]:
#remedied by creating in the new variables in the orginal df, joining TXN_value to the original df

In [494]:
del df_sn1['FID1']

In [495]:
del df_sn1['investor_id']

In [497]:
df_sn1.head()

,Txn_Val_x,Date,Units,income,riskscore,subscribed,opened,Price,return-N6M,return-HYG,...,vol_HYG,vol_IVV,TRate_y,FRSC_y,SCRate_y,gen_F,gen_M,ed_Deg or abv,ed_H.Sch/Dip.,ed_Sec. or below
0,23473.925401,2019/8/26,30.550818,324465.7233,3,0,0,86.980003,0.000000,0.000000,...,4.435599,22.102633,0.0,1.5,0.03,0,1,1,0,0
1,23473.925401,2019/12/5,0.000000,324465.7233,3,0,0,86.809998,0.000000,0.001500,...,4.435599,22.102633,0.0,1.5,0.03,0,1,1,0,0
2,23473.925401,2019/12/16,-238.537721,324465.7233,3,0,0,11.040000,-0.000905,0.001940,...,4.435599,22.102633,0.0,1.5,0.03,0,1,1,0,0
3,23473.925401,2020/3/2,120.453956,324465.7233,3,0,0,86.309998,-0.006189,0.003138,...,4.435599,22.102633,0.0,1.5,0.03,0,1,1,0,0
4,23473.925401,2020/3/25,0.000000,324465.7233,3,0,0,73.470001,0.010204,0.028128,...,4.435599,22.102633,0.0,1.5,0.03,0,1,1,0,0


**Export csv file for regression analysis**

[Pseudo code] 
- Drop the investor_id column
- Rename some of the column names that has been changed before back to its original name 

In [496]:
#no need to rename, ML model does not recognize string values

[Note on file exporting]  I have kept the original file name you have used.  However, in light of the different use cases of this Python script, I'd recommend a different naming convention to the file which goes like this:

***"Txn_Record_[Response_variable_name]_[Date_range]_[Fund_Range].csv"***

Response_variable_name:  In this case, we can use 'deltaAUM'.

Date_range: Possibly concatenate the start and end date of the slice of data.

Fund_range:  At the moment, just put 'all'.  In case we want to analyse a specific fund, filter for the relevant fund's transaction record during the earlier data preparation step and state that in the file name.

In [500]:
df_sn1.index=pd.to_datetime(df_sn1['Date'])

In [504]:
df_sn1

,Txn_Val_x,Date,Units,income,riskscore,subscribed,opened,Price,return-N6M,return-HYG,...,vol_HYG,vol_IVV,TRate_y,FRSC_y,SCRate_y,gen_F,gen_M,ed_Deg or abv,ed_H.Sch/Dip.,ed_Sec. or below
Date,,,,,,,,,,,,,,,,,,,,,
2019-08-26,23473.925401,2019/8/26,30.550818,324465.7233,3,0,0,86.980003,0.000000,0.000000,...,4.435599,22.102633,0.0,1.500000,0.030000,0,1,1,0,0
2019-12-05,23473.925401,2019/12/5,0.000000,324465.7233,3,0,0,86.809998,0.000000,0.001500,...,4.435599,22.102633,0.0,1.500000,0.030000,0,1,1,0,0
2019-12-16,23473.925401,2019/12/16,-238.537721,324465.7233,3,0,0,11.040000,-0.000905,0.001940,...,4.435599,22.102633,0.0,1.500000,0.030000,0,1,1,0,0
2020-03-02,23473.925401,2020/3/2,120.453956,324465.7233,3,0,0,86.309998,-0.006189,0.003138,...,4.435599,22.102633,0.0,1.500000,0.030000,0,1,1,0,0
2020-03-25,23473.925401,2020/3/25,0.000000,324465.7233,3,0,0,73.470001,0.010204,0.028128,...,4.435599,22.102633,0.0,1.500000,0.030000,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-23,46209.176941,2020/1/23,-1052.232630,151636.1965,3,0,0,11.180000,0.004492,-0.003057,...,4.435599,22.102633,0.0,1.666667,0.032222,0,1,0,1,0
2020-04-08,46209.176941,2020/4/8,78.010875,151636.1965,3,0,0,77.300003,0.008876,0.026015,...,4.435599,22.102633,0.0,1.666667,0.032222,0,1,0,1,0
2020-01-20,46209.176941,2020/1/20,132.608883,151636.1965,3,0,0,11.170000,0.000000,-0.001075,...,4.435599,22.102633,0.0,1.666667,0.032222,0,1,0,1,0


In [602]:
df_sn1=df_sn1.rename(columns={'Txn_Val_x':'TotalTransactions'})

In [603]:
df_trainDev=df_sn1['2019-08-26':'2020-04-01']

In [604]:
df_testing=df_sn1['2020-04-01':'2020-04-08']

In [605]:
df_trainDev.to_csv('Txn_Record_[delta_AUM]_[2019-08-26]_[2020-04-01].csv')

In [606]:
df_testing.to_csv('Txn_Record_[delta_AUM]_[2019-04-01]_[2020-04-08].csv')



**Response variable 2) Change in AUM per transaction per investor**

**Apply "Groupby" based on investor_id**

In [ ]:
keys=[df['investor_id']]

In [ ]:
grouped=df[df['trd_date'] != '2019/8/25'].groupby(keys)

In [516]:
df_avgTxn=(grouped.sum()/grouped.count())['Txn_Val']

In [519]:
df_iid2=pd.merge(df_avgTxn,df_iid,on='investor_id')

In [524]:
df_sn2=pd.merge(df_iid2,df_iid1,on='investor_id')

In [535]:
df_sn2.head()

,investor_id,Txn_Val,Date,FID1,Units,income,riskscore,subscribed,opened,Price,...,vol_HYG,vol_IVV,TRate_y,FRSC_y,SCRate_y,gen_F,gen_M,ed_Deg or abv,ed_H.Sch/Dip.,ed_Sec. or below
0,10051,2347.39254,2019/8/26,HYG,30.550818,324465.7233,3,0,0,86.980003,...,4.435599,22.102633,0.0,1.5,0.03,0,1,1,0,0
1,10051,2347.39254,2019/12/5,HYG,0.000000,324465.7233,3,0,0,86.809998,...,4.435599,22.102633,0.0,1.5,0.03,0,1,1,0,0
2,10051,2347.39254,2019/12/16,N6M,-238.537721,324465.7233,3,0,0,11.040000,...,4.435599,22.102633,0.0,1.5,0.03,0,1,1,0,0
3,10051,2347.39254,2020/3/2,HYG,120.453956,324465.7233,3,0,0,86.309998,...,4.435599,22.102633,0.0,1.5,0.03,0,1,1,0,0
4,10051,2347.39254,2020/3/25,HYG,0.000000,324465.7233,3,0,0,73.470001,...,4.435599,22.102633,0.0,1.5,0.03,0,1,1,0,0


In [526]:
del df_sn2['Txn_Val_x']

In [536]:
del df_sn2['investor_id']

In [537]:
del df_sn2['FID1']

In [596]:
df_sn2=df_sn2.rename(columns={'Txn_Val':'AverageTransaction'})

In [538]:
df_sn2.index=pd.to_datetime(df_sn2['Date'])

In [597]:
df_2TranDev=df_sn2['2019-08-26':'2020-04-01']

In [598]:
df_2Testing=df_sn2['2020-04-01':'2020-04-08']

In [599]:
df_2TranDev.to_csv('Txn_Record_[avg_AUM]_[2019-08-26]_[2020-04-01].csv')

In [600]:
df_2Testing.to_csv('Txn_Record_[avg_AUM]_[2019-04-01]_[2020-04-08].csv')

[Pseudo code] Keep only investor_id and Txn_Val of df_iid because it does not make sense to apply .sum() to the rest of the columns. 

[Pseudo code] Create df_iid2 = grouped.count().  This captures the number of transactions made by each investor.  Use the count to divide by the 'Txn_Val' field of df_iid to get a new field 'Avg_Txn_Val' - the desired response variable.  Use this new field to replace 'Txn_Val'.   

[Pseudo code] The rest of the step is the same as that for response variable 1.

**Response variable 3) Transaction count per investor**

**Apply "Groupby" based on investor_id**

In [ ]:
keys=[df['investor_id']]

In [ ]:
grouped=df[df['trd_date'] != '2019/8/25'].groupby(keys)

In [552]:
df_iid3=pd.merge(df_txnct,df_iid,on='investor_id')

In [553]:
df_iid3.columns

Index(['investor_id', 'HYG_txn_cnt_x', 'IVV_txn_cnt_x', 'N6M_txn_cnt_x',
       'Txn_Val_x', 'Date', 'FID1', 'Units', 'Gender', 'income', 'education',
       'riskscore', 'subscribed', 'opened', 'SCRate', 'TRate', 'FRSC', 'Price',
       'return-N6M', 'return-HYG', 'return-IVV', 'HYG_txn_cnt_y',
       'IVV_txn_cnt_y', 'N6M_txn_cnt_y', 'DATE', 'rtn-N6M', 'rtn-HYG',
       'rtn-IVV', 'vol_N6M', 'vol_HYG', 'vol_IVV'],
      dtype='object')

In [554]:
df_sn3=pd.merge(df_iid3,df_iid1,on='investor_id')

In [556]:
del df_sn3['HYG_txn_cnt_y']

In [557]:
del df_sn3['IVV_txn_cnt_y']

In [559]:
del df_sn3['N6M_txn_cnt_y']

In [561]:
del df_sn3['Txn_Val_x']

In [574]:
df_sn3.head()

,investor_id,HYG_txn_cnt_x,IVV_txn_cnt_x,N6M_txn_cnt_x,Date,FID1,Units,income,riskscore,subscribed,...,vol_HYG,vol_IVV,TRate_y,FRSC_y,SCRate_y,gen_F,gen_M,ed_Deg or abv,ed_H.Sch/Dip.,ed_Sec. or below
0,10051,10,10,10,2019/8/26,HYG,30.550818,324465.7233,3,0,...,4.435599,22.102633,0.0,1.5,0.03,0,1,1,0,0
1,10051,10,10,10,2019/12/5,HYG,0.000000,324465.7233,3,0,...,4.435599,22.102633,0.0,1.5,0.03,0,1,1,0,0
2,10051,10,10,10,2019/12/16,N6M,-238.537721,324465.7233,3,0,...,4.435599,22.102633,0.0,1.5,0.03,0,1,1,0,0
3,10051,10,10,10,2020/3/2,HYG,120.453956,324465.7233,3,0,...,4.435599,22.102633,0.0,1.5,0.03,0,1,1,0,0
4,10051,10,10,10,2020/3/25,HYG,0.000000,324465.7233,3,0,...,4.435599,22.102633,0.0,1.5,0.03,0,1,1,0,0


In [587]:
df_sn3['TransactionCount']=df_sn3.HYG_txn_cnt_x+df_sn3.IVV_txn_cnt_x+df_sn3.N6M_txn_cnt_x

In [588]:
del df_sn3['HYG_txn_cnt_x']

In [589]:
del df_sn3['IVV_txn_cnt_x']

In [590]:
del df_sn3['N6M_txn_cnt_x']

In [575]:
df_sn3.index=pd.to_datetime(df_sn3['Date'])

In [576]:
del df_sn3['investor_id']

In [577]:
del df_sn3['FID1']

In [591]:
df_3TranDev=df_sn3['2019-08-26':'2020-04-01']

In [592]:
df_3Testing=df_sn3['2020-04-01':'2020-04-08']

In [593]:
df_3TranDev.to_csv('Txn_Record_[count_AUM]_[2019-08-26]_[2020-04-01].csv')

In [594]:
df_3Testing.to_csv('Txn_Record_[count_AUM]_[2019-04-01]_[2020-04-08].csv')

By aggregating as .count() this already gives the desired response variable.  Add the remaining columns as per the response variable 1.